In [1]:
# -*- coding: utf-8 -*-
import sys
import os
import itertools
import numpy as np
import pandas as pd
import cobra
import matplotlib.pyplot as plt 
plt.rc('axes', axisbelow=True)

print('Python version:', sys.version)
print('numpy version:', np.__version__)
print('pandas version:', pd.__version__)
print('cobrapy version:', cobra.__version__)

Python version: 3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]
numpy version: 1.16.4
pandas version: 1.0.3
cobrapy version: 0.15.4


In [2]:
def AddRxn(model, newRxnFile):
    """Function of adding new reactions to the model."""
    n1 = len(model.reactions)
    AllAddRxn = pd.read_csv(newRxnFile, sep=',', index_col='RxnID', skipinitialspace=True)
    n2 = len(AllAddRxn)
    for i in range(n2):
        ID = AllAddRxn.index.values[i]
        addRxn = cobra.Reaction(ID)
        model.add_reactions([addRxn])
        addRxnInf = model.reactions[n1 + i]
        addRxnInf.name = AllAddRxn.loc[ID, 'RxnName']
        addRxnInf.reaction = AllAddRxn.loc[ID, 'RxnFormula']
        addRxnInf.subsystem = AllAddRxn.loc[ID, 'Subsystem']
        addRxnInf.lower_bound = AllAddRxn.loc[ID, 'LowerBound']
        addRxnInf.upper_bound = AllAddRxn.loc[ID, 'UpperBound']
    return model


In [3]:
def KORxn(model, rxns2KO):
    """Function for knocking out reactions."""
    for ID in rxns2KO:
        reaction = model.reactions.get_by_id(ID)
        reaction.knock_out()


In [4]:
def flux2file(model, product, psw, output_dir='tmp'):
    """Function of exporting flux data."""
    n = len(model.reactions)
    modelMatrix = np.empty([n, 9], dtype = object)
    for i in range(len(model.reactions)):
        x = model.reactions[i]
        modelMatrix[i, 0] = i + 1
        modelMatrix[i, 1] = x.id
        modelMatrix[i, 2] = x.name
        modelMatrix[i, 3] = x.reaction
        modelMatrix[i, 4] = x.subsystem
        modelMatrix[i, 5] = x.lower_bound
        modelMatrix[i, 6] = x.upper_bound
        modelMatrix[i, 7] = x.flux
        modelMatrix[i, 8] = abs(x.flux)
        
    df = pd.DataFrame(data = modelMatrix, 
                        columns = ['N', 'RxnID', 'RxnName', 'Reaction', 'SubSystem', 
                        'LowerBound', 'UpperBound', 'Flux-core', 'abs(Flux)'])
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    filepath = os.path.join(output_dir, '{}_{}.xlsx'.format(product, psw))
    df.to_excel(filepath, index=False)
    

## Summary of the changes in the model:

#### E. coli full model *i*ML1515 with some changes:	

* Corrected the transhydrogenase (THD2pp) to one proton translocation;
* Corrected homoserine dehydrogenase to be irreversible, towards homoserine. 
* Knocked out: ATP maintenance (ATPM), pyruvate synthase (POR5), 3pg producing glycerate kinase (GLYCK), 
pyrroloquinoline quinone (PQQ)-glucose dehydrogenase (GLCDpp), 
and the reactions for unrealistic AcCoA production DRPA & PAI2T. 
Knock out of them on top of ATPM does not change the biomass yield on glucose. 
* Deactivated pyruvate formate lyase (PFL), 2-oxobutanoate formate lyase (OBTFL).


In [5]:
model = cobra.io.load_json_model(r'..\0_ecoli_models\iML1515.json')

model.reactions.THD2pp.subtract_metabolites({"h_p": -1, "h_c": 1})
model.reactions.HSDy.bounds = (-1000, 0)

In [6]:
KORxn_base = ['POR5', 'GLYCK', 'ATPM', 'GART', 'DRPA', 'PAI2T','GLCDpp']
KORxn(model, KORxn_base)

In [7]:
# Set medium: CO2 as the sole carbon source, no limitation
for rxn in model.exchanges:
    if "C" in rxn.check_mass_balance(): # knock out all other carbon-related import
        rxn.bounds = (0, 0)
model.reactions.get_by_id('EX_co2_e').bounds = (-1000, 1000)
model.reactions.EX_meoh_e.upper_bound = 1000   # essential for 

# to avoid S as electron acceptor
# model.reactions.get_by_id('EX_h2s_e').bounds = (0,0)
# model.reactions.get_by_id('EX_met__L_e').bounds = (0,0)
# model.reactions.get_by_id('EX_cys__L_e').bounds = (0,0)

model.reactions.get_by_id('EX_fe2_e').bounds = (0,0)  # to avoid Fe as electron acceptor

# model.reactions.EX_glc__D_e.lower_bound = 0

In [8]:
model_ = model.copy()
model_ = AddRxn(model_, 'NewRxns4Full_DhapAld.csv')

unknown metabolite 'd_eryth1p_c' created
unknown metabolite 'l_eryth1p_c' created
unknown metabolite 'd_eryth4p_c' created


In [9]:
# test growth on MeOH
with model_ as m:
    m.reactions.EX_meoh_e.lower_bound = -10
    cobra.flux_analysis.pfba(m)
    m.summary()
    # flux2file(m,'Biomass','DAHPald_1',)


IN FLUXES          OUT FLUXES    OBJECTIVES
-----------------  ------------  ----------------------
meoh_e  10         h2o_e  16.8   BIOMASS_Ec_i...  0.216
o2_e     5.66      h_e     1.99
nh4_e    2.34      co2_e   1.12
pi_e     0.209
so4_e    0.0545
k_e      0.0422
fe3_e    0.00347
mg2_e    0.00188
cl_e     0.00113
ca2_e    0.00113
cu2_e    0.000153
mn2_e    0.000149
zn2_e    7.37e-05
ni2_e    6.98e-05


In [10]:
# KO DHAPT to block f6p -> dha -> dhap for better interpretation 
with model_ as m:
    m.reactions.DHAPT.knock_out()
    m.reactions.EX_meoh_e.lower_bound = -10
    cobra.flux_analysis.pfba(m)
    m.summary()
    # flux2file(m,'Biomass','DAHPald_2',)

IN FLUXES          OUT FLUXES    OBJECTIVES
-----------------  ------------  ----------------------
meoh_e  10         h2o_e  16.8   BIOMASS_Ec_i...  0.216
o2_e     5.66      h_e     1.99
nh4_e    2.34      co2_e   1.12
pi_e     0.209
so4_e    0.0545
k_e      0.0422
fe3_e    0.00347
mg2_e    0.00188
ca2_e    0.00113
cl_e     0.00113
cu2_e    0.000153
mn2_e    0.000149
zn2_e    7.37e-05
ni2_e    6.98e-05


To check the flux requirement of the pathway for the shunt, fix the bounds for biomass to 1, find the minimal flux via the shunt (like the RuMP shunt paper).  

### 1. Fully on the new cycle

In [27]:
model_.reactions.DHAPT.knock_out()

with model_ as m:
    m.reactions.EX_meoh_e.lower_bound = -1000
    KORxn(m,['FALGTHLs','EX_fald_e','EX_fe2_e'])
    m.reactions.BIOMASS_Ec_iML1515_core_75p37M.bounds = (1,1)
    m.objective = {m.reactions.DhapAld: 1}
    m.objective_direction = 'min'
    cobra.flux_analysis.pfba(m)
    m.summary()
    f_cycle = m.reactions.DhapAld.flux
    # flux2file(m,'biomass','cycle')

IN FLUXES          OUT FLUXES    OBJECTIVES
-----------------  ------------  -------------
meoh_e  46.2       h2o_e  77.8   DhapAld  46.2
o2_e    26.2       h_e     9.2
nh4_e   10.8       co2_e   5.2
pi_e     0.965
so4_e    0.252
k_e      0.195
fe3_e    0.0161
mg2_e    0.00868
cl_e     0.00521
ca2_e    0.00521
cu2_e    0.000709
mn2_e    0.000691
zn2_e    0.000341
ni2_e    0.000323
46.243173413043685


### 2. ${\Delta}$frmRAB ${\Delta}$tktAB strain:

In [12]:
# first confirm the tktAB knock out doesn't grow on glycerol
with model as m:
    m.reactions.EX_glyc_e.lower_bound = -10
    m.reactions.DHAPT.knock_out()
    m.optimize()
    m.summary()
    # methanol export is essential

IN FLUXES          OUT FLUXES    OBJECTIVES
-----------------  ------------  ----------------------
o2_e    12.5       h2o_e  32.4   BIOMASS_Ec_i...  0.521
glyc_e  10         co2_e   8.61
nh4_e    5.63      h_e     4.8
pi_e     0.503
so4_e    0.131
k_e      0.102
fe3_e    0.00837
mg2_e    0.00452
ca2_e    0.00271
cl_e     0.00271
cu2_e    0.000369
mn2_e    0.00036
zn2_e    0.000178
ni2_e    0.000168


In [13]:
with model as m:
    m.reactions.EX_glyc_e.lower_bound = -10
    m.reactions.DHAPT.knock_out()
    KORxn(m,['TKT1','TKT2'])
    m.optimize()
    m.summary()

IN FLUXES           OUT FLUXES    OBJECTIVES
------------------  ------------  ------------------
12ppd__R_e       0                BIOMASS_Ec_i...  0
12ppd__S_e       0
14glucan_e       0
15dap_e          0
23camp_e         0
23ccmp_e         0
23cgmp_e         0
23cump_e         0
23dappa_e        0
26dap__M_e       0
2ddglcn_e        0
2dglc_e          0
34dhpac_e        0
3amp_e           0
3cmp_e           0
3gmp_e           0
3hcinnm_e        0
3hpp_e           0
3hpppn_e         0
3ump_e           0
4abut_e          0
4abzglu_e        0
4hoxpacd_e       0
5dglcn_e         0
5mtr_e           0
LalaDgluMdap...  0
LalaDgluMdap_e   0
LalaDglu_e       0
LalaLglu_e       0
ac_e             0
acac_e           0
acald_e          0
acgal1p_e        0
acgal_e          0
acgam1p_e        0
acgam_e          0
acmana_e         0
acmum_e          0
acnam_e          0
acser_e          0
ade_e            0
adn_e            0
adocbl_e         0
ag_e             0
agm_e            0
akg_e        

In [33]:
# no growth, then introduce the pathway and methanol
with model_ as m:
    m.reactions.EX_meoh_e.lower_bound = -1000
    m.reactions.EX_glyc_e.lower_bound = -1000     # glycerol as co-substrate
    KORxn(m,['FALGTHLs','EX_fald_e','TKT1','TKT2'])
    m.reactions.BIOMASS_Ec_iML1515_core_75p37M.bounds = (1,1)
    m.objective = {m.reactions.DhapAld: 1}
    m.objective_direction = 'min'
    cobra.flux_analysis.pfba(m)
    m.summary()
    # flux2file(m,'biomass','TKT')
    
    f_tkt = m.reactions.DhapAld.flux
    print('\n')
    print(f'Flux requirment for the shunt: {f_tkt/f_cycle *100: .2f}%')

IN FLUXES          OUT FLUXES    OBJECTIVES
-----------------  ------------  --------------
o2_e    34.8       h2o_e  74.5   DhapAld  0.381
glyc_e  22.1       co2_e  25.7
nh4_e   10.8       h_e     9.2
pi_e     0.965
meoh_e   0.381
so4_e    0.252
k_e      0.195
fe3_e    0.0161
mg2_e    0.00868
cl_e     0.00521
ca2_e    0.00521
cu2_e    0.000709
mn2_e    0.000691
zn2_e    0.000341
ni2_e    0.000323


Flux requirment for the shunt:  0.82%


### 3. ${\Delta}$frmRAB ${\Delta}$fbp ${\Delta}$glpX strain:

In [15]:
# to confirm the fbp glpX knock out doesn't grow on glycerol
with model as m:
    m.reactions.EX_glyc_e.lower_bound = -10
    KORxn(m,['DHAPT','FBP','F6PA'])
    m.optimize()
    m.summary()

IN FLUXES           OUT FLUXES    OBJECTIVES
------------------  ------------  ------------------
12ppd__R_e       0                BIOMASS_Ec_i...  0
12ppd__S_e       0
14glucan_e       0
15dap_e          0
23camp_e         0
23ccmp_e         0
23cgmp_e         0
23cump_e         0
23dappa_e        0
26dap__M_e       0
2ddglcn_e        0
2dglc_e          0
34dhpac_e        0
3amp_e           0
3cmp_e           0
3gmp_e           0
3hcinnm_e        0
3hpp_e           0
3hpppn_e         0
3ump_e           0
4abut_e          0
4abzglu_e        0
4hoxpacd_e       0
5dglcn_e         0
5mtr_e           0
LalaDgluMdap...  0
LalaDgluMdap_e   0
LalaDglu_e       0
LalaLglu_e       0
ac_e             0
acac_e           0
acald_e          0
acgal1p_e        0
acgal_e          0
acgam1p_e        0
acgam_e          0
acmana_e         0
acmum_e          0
acnam_e          0
acser_e          0
ade_e            0
adn_e            0
adocbl_e         0
ag_e             0
agm_e            0
akg_e        

In [34]:
with model_ as m:
    m.reactions.EX_meoh_e.lower_bound = -1000
    m.reactions.EX_glyc_e.lower_bound = -1000     # glycerol as co-substrate
    KORxn(m,['FALGTHLs','EX_fald_e','FBP','F6PA'])
    m.reactions.BIOMASS_Ec_iML1515_core_75p37M.bounds = (1,1)
    m.objective = {m.reactions.DhapAld: 1}
    m.objective_direction = 'min'
    cobra.flux_analysis.pfba(m)
    m.summary()
    # flux2file(m,'biomass','FBP')

    f_fbp = m.reactions.DhapAld.flux
    print('\n')
    print(f'Flux requirment for the shunt: {f_fbp/f_cycle *100: .2f}%')

IN FLUXES          OUT FLUXES    OBJECTIVES
-----------------  ------------  -------------
o2_e    25.9       h2o_e  65     DhapAld  2.61
glyc_e  18.6       co2_e  17.4
nh4_e   10.8       h_e     9.2
meoh_e   2.61
pi_e     0.965
so4_e    0.252
k_e      0.195
fe3_e    0.0161
mg2_e    0.00868
cl_e     0.00521
ca2_e    0.00521
cu2_e    0.000709
mn2_e    0.000691
zn2_e    0.000341
ni2_e    0.000323


Flux requirment for the shunt:  5.65%


### 4. ${\Delta}$frmRAB ${\Delta}$tpiA ${\Delta}$pgk strain:

In [17]:
# to confirm the tpiA pgk knock out doesn't grow on glycerol

# first TPI
with model as m:
    m.reactions.EX_glyc_e.lower_bound = -10
    KORxn(m,['DHAPT','F6PA','MGSA','TPI','PGL'])  # TPI deletion: PGL (for PPP and ED)
    m.optimize()
    m.summary()


IN FLUXES           OUT FLUXES    OBJECTIVES
------------------  ------------  ------------------
12ppd__R_e       0                BIOMASS_Ec_i...  0
12ppd__S_e       0
14glucan_e       0
15dap_e          0
23camp_e         0
23ccmp_e         0
23cgmp_e         0
23cump_e         0
23dappa_e        0
26dap__M_e       0
2ddglcn_e        0
2dglc_e          0
34dhpac_e        0
3amp_e           0
3cmp_e           0
3gmp_e           0
3hcinnm_e        0
3hpp_e           0
3hpppn_e         0
3ump_e           0
4abut_e          0
4abzglu_e        0
4hoxpacd_e       0
5dglcn_e         0
5mtr_e           0
LalaDgluMdap...  0
LalaDgluMdap_e   0
LalaDglu_e       0
LalaLglu_e       0
ac_e             0
acac_e           0
acald_e          0
acgal1p_e        0
acgal_e          0
acgam1p_e        0
acgam_e          0
acmana_e         0
acmum_e          0
acnam_e          0
acser_e          0
ade_e            0
adn_e            0
adocbl_e         0
ag_e             0
agm_e            0
akg_e        

In [21]:
# then PGK
with model as m:
    m.reactions.EX_glc__D_e.lower_bound = -10
    m.reactions.EX_pyr_e.lower_bound = -10
    KORxn(m,['PGK','PGL','MGSA'])
    m.optimize()
    m.summary()
# only grow when fed both

IN FLUXES            OUT FLUXES    OBJECTIVES
-------------------  ------------  ----------------------
o2_e      11.2       h2o_e  17.1   BIOMASS_Ec_i...  0.384
pyr_e     10         co2_e  17
h_e        6.46
nh4_e      4.15
glc__D_e   0.463
pi_e       0.371
so4_e      0.0968
k_e        0.075
fe3_e      0.00617
mg2_e      0.00334
ca2_e      0.002
cl_e       0.002
cu2_e      0.000273
mn2_e      0.000266
zn2_e      0.000131
ni2_e      0.000124


In [35]:
with model_ as m:
    m.reactions.EX_meoh_e.lower_bound = -1000
    m.reactions.EX_glyc_e.lower_bound = -1000
    m.reactions.EX_pyr_e.lower_bound = -1000
    KORxn(m,['FALGTHLs','EX_fald_e','DHAPT','F6PA','MGSA','TPI','PGL','PGK'])
    m.reactions.BIOMASS_Ec_iML1515_core_75p37M.bounds = (1,1)
    m.objective = {m.reactions.DhapAld: 1}
    m.objective_direction = 'min'
    cobra.flux_analysis.pfba(m)
    m.summary()
    # flux2file(m, 'biomass','pgk')

    f_pgk = m.reactions.DhapAld.flux
    print('\n')
    print(f'Flux requirment for the shunt: {f_pgk/f_cycle *100: .2f}%')

IN FLUXES          OUT FLUXES    OBJECTIVES
-----------------  ------------  -------------
o2_e    52.2       co2_e  69.9   DhapAld  1.39
pyr_e   34.6       h2o_e  65.1
h_e     25.4
nh4_e   10.8
glyc_e   1.94
meoh_e   1.39
pi_e     0.965
so4_e    0.252
k_e      0.195
fe3_e    0.0161
mg2_e    0.00868
cl_e     0.00521
ca2_e    0.00521
cu2_e    0.000709
mn2_e    0.000691
zn2_e    0.000341
ni2_e    0.000323


Flux requirment for the shunt:  3.02%


In [36]:
!jupyter nbconvert --to html DhapAld_FBA.ipynb

[NbConvertApp] Converting notebook DhapAld_FBA.ipynb to html
[NbConvertApp] Writing 342985 bytes to DhapAld_FBA.html
